In [1]:
import requests
import json
import asyncio
import datetime as dt
import websockets

## Polymarket Markets Websocket API

https://docs.polymarket.com/developers/CLOB/websocket/market-channel

- PriceChangeEvent -> event_type = ”price_change”
- BookEvent -> event_type = ”book”
- LastTradePriceEvent  -> event_type = 
- TickSizeChangeEvent -> event_type = "tick_size_change"

In [55]:
from dataclasses import dataclass
from typing import Literal, List
from enum import Enum

class Side(str, Enum):
    BUY = "BUY"
    SELL = "SELL"

@dataclass
class OrderSummary:
    size: str  # or float if you want to convert from string
    price: str  # or float if you want to convert from string

@dataclass
class BookMessage:
    asset_id: str
    market: str
    timestamp: str
    hash: str
    bids: List[OrderSummary]
    asks: List[OrderSummary]
    event_type: str = "book"

@dataclass
class PriceChange:
    asset_id: str
    price: str  # can convert to float if needed
    size: str   # can convert to float if needed
    side: Side
    hash: str
    best_bid: str
    best_ask: str

@dataclass
class PriceChangeEvent:
    market: str
    price_changes: List[PriceChange]
    timestamp: str
    event_type: str = "price_change"

@dataclass
class TickSizeChangeEvent:
    asset_id: str
    market: str # condition_id
    old_tick_size: str
    new_tick_size: str
    timestamp: str
    event_type: str = "tick_size_change"

@dataclass
class LastTradePriceEvent:
    asset_id: str
    fee_rate_bps: str
    market: str # condition_id
    price: str
    side: Side
    size: str
    timestamp: str
    event_type = "last_trade_price"

In [56]:
def get_all_events(closed="false", tag_id = ''):
    params = {
        "closed": closed,
        "limit" : 500,
        "offset" : 0,
        # 'tag_id': ''
    }
    if tag_id:
        params['tag_id'] = tag_id

    events = []
    r = requests.get(url="https://gamma-api.polymarket.com/events", params=params)
    response = r.json()
    while response:
        events += response
        params['offset'] += 500
        r = requests.get(url="https://gamma-api.polymarket.com/events", params=params)
        response = r.json()
    
    return events

In [57]:
tag_15m_markets = tag_id='102467'
binary_15m_events = get_all_events(tag_id=tag_15m_markets)

In [93]:
i = 1
binary_15m_events[i]

{'id': '67271',
 'ticker': 'btc-updown-15m-1761693300',
 'slug': 'btc-updown-15m-1761693300',
 'title': 'Bitcoin Up or Down - October 28, 7:15PM-7:30PM ET',
 'description': 'This market will resolve to "Up" if the Bitcoin price at the end of the time range specified in the title is greater than or equal to the price at the beginning of that range. Otherwise, it will resolve to "Down".\nThe resolution source for this market is information from Chainlink, specifically the BTC/USD data stream available at https://data.chain.link/streams/btc-usd.\nPlease note that this market is about the price according to Chainlink data stream BTC/USD, not according to other sources or spot markets.',
 'resolutionSource': 'https://data.chain.link/streams/btc-usd',
 'startDate': '2025-10-28T20:17:27.188324Z',
 'creationDate': '2025-10-28T20:17:27.188305Z',
 'endDate': '2025-10-28T23:30:00Z',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png',
 'icon': 'https://polymarket-upl

In [94]:
asset1, asset2 = json.loads(binary_15m_events[i]['markets'][0]['clobTokenIds'])
asset3, asset4 = json.loads(binary_15m_events[i+1]['markets'][0]['clobTokenIds'])

In [95]:
asset_ids = [asset1] #, asset2, asset3, asset4]

### Test Subscribing

In [100]:
url = 'wss://ws-subscriptions-clob.polymarket.com/ws/market'
last_time_pong = dt.datetime.now()
msgs = []
async with websockets.connect(url) as websocket:
    await websocket.send(json.dumps({"assets_ids":asset_ids,"type":"market"}))

    while len(msgs) < 20:
        m = await websocket.recv()
        if m != "PONG":
            last_time_pong = dt.datetime.now()
        try:
            d = json.loads(m)
            msgs.append(d)
            # print(d)
            print(len(msgs))
        except json.JSONDecodeError:
            print("Received non-JSON message:", m)
            continue
        
        if last_time_pong + dt.timedelta(seconds=10) < dt.datetime.now():
            await websocket.send("PING")

1


ConnectionClosedError: no close frame received or sent

In [98]:
msgs

[[]]

In [14]:
type(msgs[0]), type(msgs[1])

(list, dict)

In [91]:
msgs[4]

{'market': '0x37a15e0bdee4a483107fd90b0a421172f749c35f6b1541c20c19a5475c9006ad',
 'price_changes': [{'asset_id': '38339732728655455731116499864190657114984023217962015362079232665734284716263',
   'price': '0.36',
   'size': '0',
   'side': 'BUY',
   'hash': '141eafc7d56e1fb9463b60c0c37d10b6020366b3',
   'best_bid': '0.42',
   'best_ask': '0.45'},
  {'asset_id': '55901882302422027747829623329116208925045166126595406615928499719811557157145',
   'price': '0.64',
   'size': '0',
   'side': 'SELL',
   'hash': 'cce410a81a492e46cb413ea17bcbe8930a1f9aee',
   'best_bid': '0.55',
   'best_ask': '0.58'}],
 'timestamp': '1761693623340',
 'event_type': 'price_change'}

In [26]:
from collections import defaultdict

In [77]:
def count_asset_occurrences(msgs):
    asset_id_counts = defaultdict(int)
    for msg in msgs:
        if isinstance(msg, list):
            for book in msg:
                asset_id_counts[book['asset_id']] += 1
                # print(book['asset_id'], end=", ")
        elif isinstance(msg, dict):
            if msg['event_type'] == 'price_change':
                for price_change in msg['price_changes']:
                    asset_id_counts[price_change['asset_id']] += 1
                    # print(price_change['asset_id'], end = ", ")
            elif msg['event_type'] == 'book':
                asset_id_counts[msg['asset_id']] += 1
    return asset_id_counts


In [78]:
count_asset_occurrences(msgs)

defaultdict(int,
            {'38339732728655455731116499864190657114984023217962015362079232665734284716263': 20,
             '55901882302422027747829623329116208925045166126595406615928499719811557157145': 19})

### Test Unsubscribing

# THERE IS NO FEATURE TO UNSUBSCRIBE. INSTEAD

In [89]:
url = 'wss://ws-subscriptions-clob.polymarket.com/ws/market'
last_time_pong = dt.datetime.now()
msgs1 = []
msgs2 = []
async with websockets.connect(url) as websocket:
    await websocket.send(json.dumps({"assets_ids":asset_ids,"type":"market"}))

    while len(msgs1) < 20:
        m = await websocket.recv()
        if m != "PONG":
            last_time_pong = dt.datetime.now()
        try:
            d = json.loads(m)
            msgs1.append(d)
            # print(d)
            print(len(msgs1))
        except json.JSONDecodeError:
            print("Received non-JSON message:", m)
            continue
        
        if last_time_pong + dt.timedelta(seconds=10) < dt.datetime.now():
            await websocket.send("PING")
    
    print("UNSUBBING")
    await websocket.send(json.dumps({"assets_ids":[],"type":"market"}))
    while len(msgs2) < 20:
        m = await websocket.recv()
        if m != "PONG":
            last_time_pong = dt.datetime.now()
        try:
            d = json.loads(m)
            msgs2.append(d)
            # print(d)
            print(len(msgs2))
        except json.JSONDecodeError:
            print("Received non-JSON message:", m)
            continue
        
        if last_time_pong + dt.timedelta(seconds=10) < dt.datetime.now():
            await websocket.send("PING")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
UNSUBBING
1
2
3
4
5
6
7
8
9
10
11
Received non-JSON message: PONG
12
13
14
15
16
17
18
19
20


In [90]:
count_asset_occurrences(msgs1), count_asset_occurrences(msgs2)

(defaultdict(int,
             {'38339732728655455731116499864190657114984023217962015362079232665734284716263': 20,
              '55901882302422027747829623329116208925045166126595406615928499719811557157145': 19}),
 defaultdict(int,
             {'38339732728655455731116499864190657114984023217962015362079232665734284716263': 20,
              '55901882302422027747829623329116208925045166126595406615928499719811557157145': 20}))